# Authorization Area

In [7]:
import urllib.parse
import pandas as pd
import requests
import configparser
import warnings
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [8]:
#Reading from Config file
config = configparser.ConfigParser()
config.read('config.cfg')

#api key and api secret
API_KEY = config.get('INDEX', 'api_key')
SECRET_KEY = config.get('INDEX', 'secret_key')
temp_url = config.get('INDEX', 'url')
RURL = urllib.parse.quote(temp_url,safe="") 

In [ ]:
uri = f'https://api-v2.upstox.com/login/authorization/dialog?response_type=code&client_id={API_KEY}&redirect_uri={RURL}'
uri

In [10]:
# Paste your code here
code = ''

In [11]:
URL = 'https://api-v2.upstox.com/login/authorization/token'

headers = {
    'accept':'application/json',
    'Api-Version': '2.0',
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {
    'code':code,
    'client_id': API_KEY,
    'client_secret': SECRET_KEY,
    'redirect_uri': temp_url,
    'grant_type': 'authorization_code'
}

response = requests.post(URL,headers=headers, data=data)
json_response = response.json()

In [12]:
with open("../csv/authentication.json", "w") as outfile:
    json.dump(json_response, outfile)

In [ ]:
# Opening JSON file
with open('../csv/authentication.json', 'r') as openfile:
    json_response = json.load(openfile)

ACCESS_TOKEN = json_response['access_token']
print(ACCESS_TOKEN)

------------------------------------------Autherization Completed---------------------------------------------------------------------------------------

# Generating Instrument Files

In [15]:
instruments_url = 'https://assets.upstox.com/market-quote/instruments/exchange/complete.csv.gz'
all_symbols = pd.read_csv(instruments_url)
all_symbols.to_csv('../csv/all_symbols.csv')

In [16]:
all_symbols = pd.read_csv('../csv/all_symbols.csv', index_col = 0)
all_symbols['expiry'] = pd.to_datetime(all_symbols['expiry']).apply(lambda x: x.date())
print(f'Shape Of DataFrame: {all_symbols.shape}')
all_symbols.head()

Shape Of DataFrame: (87513, 12)


,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,19819.95,NaT,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,23526.35,NaT,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,23384.40,NaT,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NIFTY100 LowVol30,NaN,NaN,NIFTY100 LowVol30,15179.45,NaT,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|NIFTY Alpha 50,NaN,NaN,NIFTY Alpha 50,36197.55,NaT,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX


------------------------------------------All Symbols Generated---------------------------------------------------------------------------------------

# Creating a protobuf py file

In [16]:
import requests
import subprocess
from google.protobuf.compiler import plugin_pb2 as plugin

In [17]:
def download_proto_file(url, save_path):
    try:
        response = requests.get(url)
        response.raise_for_status()

        with open(save_path, 'wb') as file:
            file.write(response.content)

        print(f"Proto file downloaded and saved to: {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading Proto file: {e}")

def generate_python_from_proto(proto_file_path, output_dir):
    try:
        # Run the protoc command to generate Python code
        subprocess.run(['protoc', '--python_out', output_dir, proto_file_path], check=True)
        print(f"Python code generated from Proto file: {proto_file_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error generating Python code from Proto file: {e}")

def download_and_generate_proto(url, proto_save_path, python_output_dir):
    try:
        download_proto_file(url, proto_save_path)
        generate_python_from_proto(proto_save_path, python_output_dir)
    except Exception as e:
        print(f"An error occurred: {e}")

In [18]:
# Replace these with your actual URLs, file paths, and directories
proto_url = 'https://assets.upstox.com/feed/market-data-feed/v1/MarketDataFeed.proto'
proto_save_path = './MarketDataFeed.proto'
python_output_dir = '../main_folder'
download_and_generate_proto(proto_url, proto_save_path, python_output_dir)
# generate_python_from_proto(proto_save_path, python_output_dir)

Proto file downloaded and saved to: ./MarketDataFeed.proto
Python code generated from Proto file: ./MarketDataFeed.proto


# Options Trading

In [69]:
import trade_module
import api_methods
import importlib
import time
import generate_symbols
import exit_strategies

In [ ]:
importlib.reload(trade_module)
importlib.reload(api_methods)
importlib.reload(generate_symbols)
importlib.reload(exit_strategies)

Indx = 'NSE_INDEX|Nifty 50' 
expiry_week = 1
level= '1-OTM'
ce_pe = 'PE'
tradingsymbol = generate_symbols.get_options_trading_symbol(Indx = Indx, expiry_week = expiry_week, level= level, ce_pe = ce_pe)
quantity = 50

print(tradingsymbol)

instrument = all_symbols.dropna(subset = ['tradingsymbol'],inplace = False)[(all_symbols.dropna(subset = ['tradingsymbol'],inplace = False)['tradingsymbol'].str.startswith(tradingsymbol))]['instrument_key'].values[0]

print(instrument,quantity)
print(f'**YOU ARE PLACING {ce_pe} ORDER**')

In [144]:
# Run the below cell to create instance for the 
trade_2 = trade_module.Trade(instrument,quantity)

In [ ]:
# Run the function from the exitstrategy class this is the strategfy to exit trade  you just have to run this cell when you want to enter a trade an you will exit the trade according to the strategy
# make sure your websocket is working properly before entring trade
strategy = exit_strategies.exit_strategy(trade_2,sl_percent = 0.005,target_percent = 0.005,factor = 1.01,big_move=False)
strategy.buy_trail_stop_loss()